In [13]:
import os

import numpy as np
import pandas as pd

from utils import constants

# Linear SVM Experiments

In [45]:
root_dir = os.path.join(constants.RESULTS_DIR, 'motor_LR', 'baseline', 'linear_svm')

exp_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

df = pd.DataFrame()

for d in exp_dirs:
    if d.startswith('no_select'):
        window = '_'.join(d.split('_')[4:])
        temp = pd.read_parquet(os.path.join(root_dir, d, 'None_results.parquet'))
        temp.loc[:, 'selection_method'] = 'None'
        temp.loc[:, 'window'] = window
    else:
        selection_method = d.split('_')[-1]
        window = '_'.join(d.split('_')[2:-1])
        if selection_method in ['mi', 'pca', 'rfe', 'sfs']:
            selection_method = selection_method.upper()
        temp = pd.read_parquet(os.path.join(root_dir, d, f'{selection_method}_results.parquet'))
        temp.loc[:, 'selection_method'] = selection_method
        temp.loc[:, 'window'] = window
    df = df.append(temp, ignore_index=True)

metrics = ['Accuracy', 'Precision', 'Recall', 'F1']
train_cols = [f'Train {m}' for m in metrics]
valid_cols = [f'Valid {m}' for m in metrics]
test_cols = [f'Test {m}' for m in metrics]

# Sort by column values
window_sorter = ['all', 'pre_stim', 'init', 'pre_rt', 'rt', 'post_rt']
select_sorter = ['None', 'PCA', 'MI']
df.window = df.window.astype('category')
df.window.cat.set_categories(window_sorter, inplace=True)
df.selection_method = df.selection_method.astype('category')
df.selection_method.cat.set_categories(select_sorter, inplace=True)
df.sort_values(['window', 'selection_method'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [46]:
df.groupby(['window', 'selection_method']).mean()[test_cols]

Test Accuracy  Test Precision  Test Recall  \
window   selection_method                                               
all      None                   0.489952        0.489603     0.490053   
         PCA                    0.490287        0.490176     0.490488   
         MI                     0.491031        0.490667     0.490970   
pre_stim None                   0.488855        0.488449     0.488770   
         PCA                    0.488788        0.488630     0.488859   
         MI                     0.499002        0.499051     0.498927   
init     None                   0.497253        0.497125     0.497299   
         PCA                    0.493721        0.493572     0.493762   
         MI                     0.491902        0.491622     0.491821   
pre_rt   None                   0.506677        0.506708     0.506605   
         PCA                    0.501949        0.501814     0.501981   
         MI                     0.500036        0.499946     0.499947   
rt       None                   0.495812        0.495675     0.495645   
         PCA                    0.498536        0.498418     0.498643   
         MI                     0.493493        0.493088     0.493315   
post_rt  None                   0.490667        0.490311     0.490730   
         PCA                    0.493747        0.493591     0.493790   
         MI                     0.490296        0.489945     0.490237   

                            Test F1  
window   selection_method            
all      None              0.486257  
         PCA               0.487125  
         MI                0.487402  
pre_stim None              0.485260  
         PCA               0.485589  
         MI                0.495542  
init     None              0.493283  
         PCA               0.490077  
         MI                0.487945  
pre_rt   None              0.503344  
         PCA               0.498509  
         MI                0.496420  
rt       None              0.492510  
         PCA               0.495328  
         MI                0.490300  
post_rt  None              0.486790  
         PCA               0.490142  
         MI                0.486691